In [0]:
import pandas as pd
import numpy as np
import sys
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Importing Datasets

In [0]:
dfInteractions = pd.read_csv(
   'data/interactions.tsv',
   # sep='\t',
   error_bad_lines=False,
   low_memory=False
)
print('sucesso')

In [0]:
dfInteractions.head(10)

In [0]:
dfInteractions.shape

In [0]:
dfInteractions.columns

In [0]:
dfVertebrates = pd.read_csv(
   '/data/AllVertebrates.csv,
   # sep='\t',
   error_bad_lines=False,
   low_memory=False
)
print('sucesso')

In [0]:
dfVertebrates.head(10)

In [0]:
dfVertebrates.columns

In [0]:
dfVertebrates.head()
dfVertebrates.count()

# Normalizing Species

## Vertebrates

In [0]:
#https://stackoverflow.com/questions/36383821/pandas-dataframe-apply-function-to-column-strings-based-on-other-column-value
dfVertebrates['final_name'] = dfVertebrates.apply(lambda L: L.Genus.strip().lower() + ' ' + L.Species.strip().lower(), axis=1)

dfVertebrates.head()

#dfVertebrates.count()
arrSpeciesVertebrates = dfVertebrates['final_name'].unique()

dfSpeciesVertebrates = pd.DataFrame({'final_name':arrSpeciesVertebrates[:]})

dfSpeciesVertebrates.head(10)

In [0]:
#execute the following to test
dfSpeciesVertebrates.count()
#expected 46557

##Interactions

In [0]:
# https://stackoverflow.com/questions/22005911/convert-columns-to-string-in-pandas
# df[["D", "E"]] = df[["D", "E"]].astype(int) 

#getting only the columns that matters now (taxon name)
dfInteractions[['sourceTaxonName','targetTaxonName']] = dfInteractions[['sourceTaxonName','targetTaxonName']].astype(str)

dfInteractions['source_final_name'] = dfInteractions.apply(lambda L: L.sourceTaxonName.strip().lower(), axis=1)
dfInteractions['target_final_name'] = dfInteractions.apply(lambda L: L.targetTaxonName.strip().lower(), axis=1)

dfInteractions[['sourceTaxonName','source_final_name','targetTaxonName','target_final_name']].head(10)

In [0]:
#way2
#interactionsTaxonNamesWithoutNaN = pd.concat([dfInteractions[dfInteractions.targetTaxonSpeciesName.isnull()], dfInteractions[dfInteractions.targetTaxonSpeciesName.notnull()]])

In [0]:
arrSpeciesInteractions = pd.concat([dfInteractions['source_final_name'], dfInteractions['target_final_name']])

arrSpeciesInteractions.head(10)

In [0]:
#execute the following to test
arrSpeciesInteractions.count()
#expected 7331620

In [0]:
arrSpeciesInteractions = arrSpeciesInteractions.unique()

dfSpeciesInteractions = pd.DataFrame({'final_name':arrSpeciesInteractions[:]})

print('sucesso')

In [0]:
dfSpeciesInteractions.head(10)

In [0]:
#execute the following to test
dfSpeciesInteractions.count()
#expected 211397

## Intersection

In [0]:
dfInterSpecies = pd.merge(dfSpeciesInteractions, dfSpeciesVertebrates, on="final_name")

dfInterSpecies.head()

In [0]:
#execute the following to test
dfInterSpecies.count()
#expected 13626

## All Species

In [0]:
dfAllSpecies = pd.concat([dfSpeciesInteractions, dfSpeciesVertebrates])

dfAllSpecies.head(10)

In [0]:
#execute the following to test
dfAllSpecies.count()
#expected 257954

In [0]:
dfAllSpecies = dfAllSpecies['final_name'].unique()

dfAllSpecies = pd.DataFrame({'final_name':dfAllSpecies[:]})

dfAllSpecies = dfAllSpecies.sort_values(by=['final_name'])

print('sucesso')

In [0]:
dfAllSpecies.head(10)

In [0]:

dfInteractions[dfInteractions.targetTaxonName=='1 of 100']

In [0]:
#execute the following to test
dfAllSpecies.head(10)
#expected 244328

# Joining

Will consist in Join the following sequence:

IUCN source info

Interactions

IUCN target info

In [0]:
#https://stackoverflow.com/questions/44768808/python-pandas-merging-two-dataframes-based-in-a-non-index-column
#df1.merge(df2,  on='cnpj',suffixes=('','_y')).drop('index_y',axis=1)

#LONG VERSION
#dfFinalJoin = pd.merge(dfAllSpecies, dfVertebrates, left_on='final_name',right_on='final_name', how='inner')

#SHORT VERSION
#dfFinalJoin = pd.merge(dfAllSpecies, dfVertebrates, on='final_name')


dfSourceJoin = pd.merge(dfVertebrates, dfInteractions, left_on='final_name',right_on='source_final_name', how='inner')


dfSourceJoin.head(10)


In [0]:
#checking size of JOIN from IUCN to source Interactions
dfSourceJoin.shape
#Expected 1.124.939

In [0]:
#JUST TO KNOW, THE RIGHT SIZE

dfTargetJoin = pd.merge(dfInteractions, dfVertebrates, left_on='target_final_name',right_on='final_name', how='inner')

#checking size of JOIN from IUCN to target Interactions
dfTargetJoin.shape
#553534

In [0]:
dfFinalJoin = pd.merge(dfSourceJoin, dfVertebrates, left_on='target_final_name', right_on='final_name', suffixes=("_source","_target"), how='inner')


dfFinalJoin.head(10)


In [0]:
#getting final join n of rows
dfFinalJoin.shape
#should be 119.504

In [0]:
export_csv = dfFinalJoin.to_csv ('data/interactions_between_vertebrates.csv', index = None, header=True)

print('sucesso')

In [0]:
dfFinalEats = dfFinalJoin[dfFinalJoin['interactionTypeName']=="eats"]
dfFinalEats = dfFinalEats.reset_index()
dfFinalEats = pd.concat([dfFinalEats["source_final_name"], dfFinalEats["target_final_name"]])



dfFinalEats.unique().shape

# Conclusions


(2019 October 15th, 21h44m)

In Vertebrates dataset, there are 46.557 registers, with 46.557 unique species

In Interactions dataset, there are 3.665.810 registers, with 211.397 unique species

When matching datasets, we get 257.954 registers, with 244.328 unique species.

That means that we have 13.626 intersection matches. That represents 29.26% from the Vertebrates dataset

(2019 October 19th, 23h59m)

Officially, after joining the bases, we got 13.626 unique species found in both datasets.

When Joining, we had 1.124.939 interactions that had a source at IUCN Vertebrates dataset, and 553.534 interactions that had a target at IUCN Vertebrates dataset

At last, we got 119.504 interactions that had both a source and target that was at IUCN Vertebrates dataset